## Vector stores and retrievers

this tutorial will familiarize you with lanfchais vector  store and retriever abstractions. these absrractions are designed to support  retrieval of data -- from (vector) database and other sources -- for integratins wiht llm workflow.they are impottant for applicstins that fetch data to be reasomed over as part of modle inerence as in the retrieval augmented generation.

we will cover

docments

vector stores

retrievers


In [1]:
from langchain_core.documents import Document
documents = [
    Document(
        page_content="Dogs are great companions known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"}
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"}
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"}
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"}
    )
]

In [2]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [24]:
import os 
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq
groq_api_key=os.getenv("groq_api_key")

os.environ["hf_token"]=os.getenv('hf_token')
llm=ChatGroq(api_key=groq_api_key,model="llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000178C6782290>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000178C5E5DC90>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embedding=HuggingFaceEmbeddings(model_name="all-miniLM-L6-v2")
#emedding.embed(documents)

In [8]:
## vetor store =convert data into vector and store
from langchain_chroma import Chroma


vector_store=Chroma.from_documents(documents=documents,embedding=embedding)
vector_store

In [9]:
vector_store.similarity_search("cat")

[Document(id='696e73a6-d269-4fe7-900f-758429baa41c', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='ff9fc6cf-9c64-43c4-8199-95e5de777127', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='914056d4-3f67-484e-8487-6d905bf7067e', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions known for their loyalty and friendliness.'),
 Document(id='46b4c935-fd28-4820-a6b7-6eb8fb04bdbd', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions known for their loyalty and friendliness.')]

In [10]:
## Async query
await vector_store.asimilarity_search("cat")

[Document(id='696e73a6-d269-4fe7-900f-758429baa41c', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='ff9fc6cf-9c64-43c4-8199-95e5de777127', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='914056d4-3f67-484e-8487-6d905bf7067e', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions known for their loyalty and friendliness.'),
 Document(id='46b4c935-fd28-4820-a6b7-6eb8fb04bdbd', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions known for their loyalty and friendliness.')]

###

In [20]:
retriever=vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}

)
invoke_m=retriever.invoke("cat")
#print(invoke_m)
retriever.batch(["cat","dog"])
#print(batch_m)

[[Document(id='696e73a6-d269-4fe7-900f-758429baa41c', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='46b4c935-fd28-4820-a6b7-6eb8fb04bdbd', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions known for their loyalty and friendliness.')]]

In [26]:
# this is RAG retrival argumented generated

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message=""" 
    answer the question using the provided context only

    {question}

    context:
    {context}
"""

prompt=ChatPromptTemplate.from_messages([("human",message)])



rag_chain={"context":retriever,"question":RunnablePassthrough()}|prompt|llm
response=rag_chain.invoke("tell me about dogs")
print(response.content)

Dogs are great companions known for their loyalty and friendliness.
